In [1]:
import mlrun

In [2]:
# nuclio: start-code

In [3]:
%nuclio cmd -c pip install lightgbm

In [4]:
%nuclio config spec.build.baseImage = "mlrun/mlrun"
%nuclio config kind = "serving"

%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'
%nuclio: setting kind to 'serving'


In [5]:
import numpy as np 
import pandas as pd
import lightgbm as lgbm
from mlrun.artifacts import get_model, update_model

In [6]:
import mlrun
import numpy as np
import lightgbm as lgbm
import os
from cloudpickle import load

class LGBMModel(mlrun.serving.V2ModelServer):
    
    def load(self):
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body):
        try:
            feats = np.asarray(body['inputs'])
            result = self.model.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

In [7]:
# nuclio: end-code

In [8]:
models_path = 'https://s3.wasabisys.com/iguazio/models/lightgbm/SampleModel.pkl'

In [9]:
fn = mlrun.code_to_function('lightgbm-serving',
                            description="LightGBM Serving",
                            categories=['serving', 'ml'],
                            labels={'author': 'edmondg', 'framework': 'lightgbm'},
                            code_output='.')
fn.spec.default_class = 'LGBMModel'

In [10]:
fn.add_model('nyc-taxi-server', model_path=models_path)

In [11]:
fn.apply(mlrun.platforms.auto_mount())
address = fn.deploy()

> 2021-01-27 09:58:53,966 [info] Starting remote function deploy
2021-01-27 09:58:54  (info) Deploying function
2021-01-27 09:58:54  (info) Building
2021-01-27 09:58:54  (info) Staging files and preparing base images
2021-01-27 09:58:54  (info) Building processor image
2021-01-27 09:58:55  (info) Build complete
2021-01-27 09:59:03  (info) Function deploy complete
> 2021-01-27 09:59:04,345 [info] function deployed, address=default-tenant.app.jinkwubtllaf.iguazio-cd1.com:32617


In [12]:
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 3, 5.1, 3.5, 1.4, 0.2, 5.1, 3.5, 1.4, 0.2, 5.1, 3.5, 1.4, 0.2]]}'''
fn.invoke('/v2/models/nyc-taxi-server/predict', my_data)

{'id': '10ad6a7b-fcc5-4d38-a2d5-a77a6e4efe18',
 'model_name': 'nyc-taxi-server',
 'outputs': [25.374309065093435]}